In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_log_error
from statsmodels.tsa.ar_model import AR
from xgboost import XGBRegressor
from datetime import datetime
from datetime import timedelta
from ortools.linear_solver import pywraplp
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
import common
%matplotlib inline

Let us try to see demand of one product in one region

Hmm... There does not seem to be any significant correlation at the product-region level. Atleast for this combination. Let us try to plot the aggregate demand per product per month

This does not seem to have much auto correlation either. Finally let us try the entire product line and see if there is anything there

Hmm. We will have to try different algorithms to check what performs better for this dataset. First let us get the accuracy function as specified in the contest. This does a overall average as against the monthly average, but that should be fine.

In [5]:
class ARWrapper():
    def __init__(self,silent=False):
        self.model = None
        self.model_fit = None
        self.silent = silent
    def fit(self,train):
        self.train_len = len(train)
        self.model = AR(train,freq='Q')
        self.model_fit = self.model.fit(ic='bic',maxiter=100)
        if(self.silent==False):
            print('Lag: %s' % self.model_fit.k_ar)
            print('Coefficients: %s' % self.model_fit.params)        
    def predict(self,test_len):
        pred = self.model_fit.predict(start=self.train_len,end=self.train_len+test_len-1,dynamic=False)
        return pred

my_ar = ARWrapper(silent=True)
my_ar.fit([1,2,3,4,1,2,3,4,1,2,3,4,1,2,3,4])
print(my_ar.predict(10))

[ 1.  2.  3.  4.  1.  2.  3.  4.  1.  2.]


The accuracy is pathetic with the AR model. As we saw anectodically prior to running the model, current demand does not seem to depend on previous values. Let us try some other non-time-series models. 

In [76]:
def fit_and_predict_non_ts(train,test):
    train['pid'] = train.Product_ID.apply(lambda x:int(x[1:]))
    train['rid'] = train.Region.apply(lambda x:int(x[1:]))
    test['pid'] = test.Product_ID.apply(lambda x:int(x[1:]))
    test['rid'] = test.Region.apply(lambda x:int(x[1:]))
    train = train.sort_values(['Month','pid','rid'])
    test = test.sort_values(['Month','pid','rid'])
    combined = pd.concat([train,test],axis=0)
    combined['d3'] = combined.Demand.shift(3*18*81)
    combined['d4'] = combined.Demand.shift(4*18*81)
    combined['d5'] = combined.Demand.shift(5*18*81)
    combined['d6'] = combined.Demand.shift(6*18*81)
#     combined['d7'] = combined.Demand.shift(7*18*81)
#     combined['d8'] = combined.Demand.shift(8*18*81)    
#     combined['d9'] = combined.Demand.shift(9*18*81)    
#     combined['d10'] = combined.Demand.shift(10*18*81)    
#     combined['d11'] = combined.Demand.shift(11*18*81)    
#     combined['d12'] = combined.Demand.shift(12*18*81)    
#     combined['d13'] = combined.Demand.shift(13*18*81)    
    train = combined.iloc[:len(train)].copy()
    test = combined.iloc[len(train):].copy()
    exclude_features=['Product_ID','Region','Demand','Month']
    model = XGBRegressor()
    #model = LinearRegression()
    vtrain = train
    #vtrain = train.loc[train.d13.notnull()]
    model.fit(vtrain[list(set(vtrain.columns)-set(exclude_features))],vtrain.Demand)
    pred = model.predict(test[list(set(test.columns)-set(exclude_features))])
    test['Demand'] = pred
    return test
    
    
    #bfhd = pd.concat([bfhd,pd.get_dummies(bfhd.Product_ID)],axis=1)
    #bfhd = pd.concat([bfhd,pd.get_dummies(bfhd.Region)],axis=1)

#     bfhd = bfhd.sort_values(['pid','rid'])
#     exclude_features=['Product_ID','Region','Demand','Month']

#     trainX = bfhd[bfhd.Month < 34].copy()
#     valX = bfhd[bfhd.Month >=34].copy()
#     model = XGBRegressor()
#     model.fit(trainX[list(set(trainX.columns)-set(exclude_features))],trainX.Demand)
#     pred = model.predict(valX[list(set(valX.columns)-set(exclude_features))])
#     print("Accuracy = ",forecast_accuracy(pd.Series(valX.Demand),pd.Series(pred)))
#     print("MAPE = ",mape(pd.Series(valX.Demand),pd.Series(pred)))
#     print("MSE = ",mean_squared_error(pd.Series(valX.Demand),pd.Series(pred)))
#    return test

In [9]:
def wape(actual,pred):
    monthly_accuracies=[]
    for month in actual.Month.unique():
        pactual = actual.loc[(actual.Demand>0)&(actual.Month==month),:]
        ppred = pred.loc[(actual.Demand>0)&(actual.Month==month),:]
        monthly_total = pactual.Demand.sum()
        perr = np.abs(pactual.Demand-ppred.Demand)/pactual.Demand
        weight = pactual.Demand/monthly_total
        weighted_mape = perr*weight
        monthly_accuracy = (1-weighted_mape).mean()
        #print(month,monthly_total,monthly_accuracy)
        monthly_accuracies.append(monthly_accuracy)
    mean_monthly_accuracy = np.mean(monthly_accuracies)
    total_accuracy = mean_monthly_accuracy*100
    return abs(total_accuracy.round(2))

def total_accuracy(actual,pred):
    actual = actual.sort_values(['Product_ID','Region','Month'])
    pred = pred.sort_values(['Product_ID','Region','Month'])
    monthly_accuracies=[]
    for month in actual.Month.unique():
        pactual = actual.loc[(actual.Demand>0)&(actual.Month==month),:]
        ppred = pred.loc[(actual.Demand>0)&(actual.Month==month),:]
        monthly_total = pactual.Demand.sum()
        weight = pactual.Demand/monthly_total
        abs_err =np.abs(pactual.Demand-ppred.Demand)
        monthly_mape = sum(weight*abs_err)/sum(weight*pactual.Demand)
        monthly_accuracy = max(1-monthly_mape,0)
        #print(month, monthly_total,monthly_accuracy)
        monthly_accuracies.append(monthly_accuracy)
    mean_monthly_accuracy = np.mean(monthly_accuracies)
    total_accuracy = mean_monthly_accuracy*100
    return abs(total_accuracy.round(2))
        
def gen_df(val_fn):
    products=[]
    regions=[]
    months=[]
    demands=[]
    for product in ['P1','P2','P3']:
        for region in ['R1','R2','R3']:
            for month in range(37,40):
                products.append(product)
                regions.append(region)
                months.append(month)
                demands.append(val_fn(product,region,month))
    df = pd.DataFrame({'Product_ID':products,'Region':regions,'Month':months,'Demand':demands})
    df = df[['Product_ID','Region','Month','Demand']]
    return df

print('1,1   - ',total_accuracy(gen_df(lambda a,b,c:1),gen_df(lambda a,b,c,:1)))
print('1,0   - ',total_accuracy(gen_df(lambda a,b,c:1),gen_df(lambda a,b,c,:0)))
print('1,0.5 - ',total_accuracy(gen_df(lambda a,b,c:1),gen_df(lambda a,b,c,:0.5)))
print('1,5 - ',total_accuracy(gen_df(lambda a,b,c:1),gen_df(lambda a,b,c,:5)))

#df = gen_df(lambda a,b,c:np.random.randint(0,10))
print('1,1   - ',wape(gen_df(lambda a,b,c:1),gen_df(lambda a,b,c,:1)))
print('1,0   - ',wape(gen_df(lambda a,b,c:1),gen_df(lambda a,b,c,:0)))
print('1,0.5 - ',wape(gen_df(lambda a,b,c:1),gen_df(lambda a,b,c,:0.5)))
print('1,5 - ',wape(gen_df(lambda a,b,c:1),gen_df(lambda a,b,c,:5)))



1,1   -  100.0
1,0   -  0.0
1,0.5 -  50.0
1,5 -  0.0
1,1   -  100.0
1,0   -  88.89
1,0.5 -  94.44
1,5 -  55.56


In [10]:
def fit_and_predict(train,test):
    products =[]
    regions=[]
    for i in range(1,82):
        products.append('P'+str(i))
    for i in range(1,19):
        regions.append('R'+str(i))
    
    train = train.sort_values(['Product_ID','Region'])
    test = test.sort_values(['Product_ID','Region'])
    for product in products:
        for region in regions:
            strain = train[(train.Product_ID==product)&(train.Region==region)]
            stest = test[(test.Product_ID==product)&(test.Region==region)]
            train_demand = strain.Demand.values
            ar = ARWrapper(silent=True)
            ar.fit(train_demand)
            pred = ar.predict(3)
            test.loc[(test.Product_ID==product)&(test.Region==region),'Demand']=pred
    return test

In [77]:
his_dem = pd.read_csv('historical_demand.csv')
for till_month in range(24,36,3):
    train = his_dem.loc[his_dem.Month <= till_month,:].copy()
    test = his_dem.loc[(his_dem.Month > till_month) & (his_dem.Month <= till_month+3),:].copy()
    test_copy = test.copy()
    test.Demand = 0
    pred = fit_and_predict_non_ts(train,test)
    #pred = fit_and_predict(train,test)
    print("Train 1-{} Test {}-{} Accuracy: {} MSE:{}".format(till_month,till_month+1,till_month+3,
                total_accuracy(test_copy,pred),mean_squared_error(test_copy.Demand,pred.Demand)))
    

Train 1-24 Test 25-27 Accuracy: 91.24 MSE:86297.6672799045
Train 1-27 Test 28-30 Accuracy: 76.19 MSE:130229.00928539019
Train 1-30 Test 31-33 Accuracy: 67.72 MSE:75075.52227497667
Train 1-33 Test 34-36 Accuracy: 61.81 MSE:70890.58680547877
